Text Classification with ML algorithms : MLP, LogisticRegression, SVM, KNN, Random Forest

In [10]:


import matplotlib.pyplot as plt
import os
import re
import string
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pydot
import shutil
import string
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
print(tf.__version__)
#policy = mixed_precision.Policy('mixed_float16')
#mixed_precision.set_policy(policy)

2.9.2


In [12]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [13]:
# from google.colab import files
# uploaded = files.upload()
df = pd.read_csv("mtsamples.csv")
df.head()

,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


In [14]:
df=df[['transcription','medical_specialty']]
df.head()

,transcription,medical_specialty
0,"SUBJECTIVE:, This 23-year-old white female pr...",Allergy / Immunology
1,"PAST MEDICAL HISTORY:, He has difficulty climb...",Bariatrics
2,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...",Bariatrics
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary


In [15]:
df = df[df['medical_specialty'] != ' Surgery']
df = df[df['medical_specialty'] != ' SOAP / Chart / Progress Notes']
df = df[df['medical_specialty'] != ' Emergency Room Reports']
df = df[df['medical_specialty'] != ' Discharge Summary']
df = df[df['medical_specialty'] != ' Office Notes']
df = df[df['medical_specialty'] != ' General Medicine']
df = df[df['medical_specialty'] != ' Pain Management']

In [16]:
df.isna().sum()

transcription        16
medical_specialty     0
dtype: int64

In [17]:
df.dropna(axis=0,inplace=True)

In [18]:
counts = df['medical_specialty'].value_counts()

df = df[~df['medical_specialty'].isin(counts[counts < 240].index)]
df.head()

,transcription,medical_specialty
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary
7,"2-D ECHOCARDIOGRAM,Multiple views of the heart...",Cardiovascular / Pulmonary
9,"DESCRIPTION:,1. Normal cardiac chambers size....",Cardiovascular / Pulmonary
11,"2-D STUDY,1. Mild aortic stenosis, widely calc...",Cardiovascular / Pulmonary


In [19]:
special_character_remover = re.compile('[/(){}\[\]\|@,;]')
extra_symbol_remover = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

Lemmatization is the process of converting a word to its base form. The difference between stemming and lemmatization is, lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, often leading to incorrect meanings and spelling errors.

Tokenization is used in natural language processing to split paragraphs and sentences into smaller units that can be more easily assigned meaning.   

The first step of the NLP process is gathering the data (a sentence) and breaking it into understandable parts (words).

In [20]:
def clean_text(text):
    text = text.lower()
    text = special_character_remover.sub(' ',text)
    text = extra_symbol_remover.sub('',text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

def lemmatize_text(text):
    wordlist=[]
    lemmatizer = WordNetLemmatizer() 
    sentences=sent_tokenize(text)
    
    for sentence in sentences:
        words=word_tokenize(sentence)
        for word in words:
            wordlist.append(lemmatizer.lemmatize(word))    
    return ' '.join(wordlist) 

In [21]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [22]:
df['transcription'] = df['transcription'].apply(clean_text)
df['transcription'] = df['transcription'].apply(lemmatize_text)

Whenever we apply any algorithm to textual data, we need to convert the text to a numeric form. Hence, there arises a need for some pre-processing techniques that can convert our text to numbers. Both bag-of-words (BOW) and TFIDF are pre-processing techniques that can generate a numeric form from an input text.

Bag-of-Words:
The bag-of-words model converts text into fixed-length vectors by counting how many times each word appears.

In [23]:
vectorizer = TfidfVectorizer(analyzer='word', stop_words='english',ngram_range=(1,3), max_df=0.75,min_df=5, use_idf=True, smooth_idf=True,sublinear_tf=True, max_features=1000)
tfIdfMat  = vectorizer.fit_transform(df['transcription'].tolist() )
feature_names = sorted(vectorizer.get_feature_names())
del feature_names[0:35]
print(feature_names)

['60', '6french', '70', '75', '80', '90', '92', '93', '95', '96', 'abc', 'abcd', 'abdomen', 'abdomen soft', 'abdomen soft nontender', 'abdominal', 'abdominal pain', 'able', 'abnormal', 'abnormality', 'abuse', 'according', 'achieved', 'active', 'activity', 'acute', 'acute distress', 'addition', 'additional', 'adenopathy', 'adequate', 'administered', 'admission', 'admitted', 'advanced', 'affect', 'age', 'ago', 'air', 'airway', 'alcohol', 'alert', 'alert oriented', 'allergic', 'allergy', 'allergy known', 'allergy known drug', 'anemia', 'anesthesia', 'anesthesia general', 'anesthetic', 'angiogram', 'angiography', 'ankle', 'anterior', 'anterior descending', 'antibiotic', 'anxiety', 'aorta', 'aortic', 'aortic valve', 'ap', 'apparently', 'appear', 'appearance', 'appeared', 'appears', 'applied', 'appropriate', 'approximately', 'area', 'arm', 'arterial', 'artery', 'artery disease', 'arthritis', 'asked', 'aspect', 'aspirin', 'assessment', 'associated', 'atraumatic', 'atrial', 'atrial fibrillatio

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [24]:
#Principal component analysis (PCA).
pca = PCA(n_components=0.95)
tfIdfMat_reduced = pca.fit_transform(tfIdfMat.toarray())
labels = df['medical_specialty'].tolist()
category_list = df.medical_specialty.unique()

In [25]:
# df['length']= df.transcription.str.len()


In [26]:
df['medical_specialty'] = df['medical_specialty'].map({' Cardiovascular / Pulmonary': 0, ' Consult - History and Phy.' :1, ' Gastroenterology':2, ' Neurology':3, ' Obstetrics / Gynecology': 4, ' Orthopedic':5, ' Radiology':6, ' Urology':7})

In [27]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test= train_test_split(df['transcription'], df['medical_specialty'], test_size = 0.2, random_state = 0)

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(stop_words= 'english')
tf_v_matrix_train = vec.fit_transform(x_train)
tf_v_matrix_valid = vec.transform(x_test)
tf_v_matrix_train.shape

(1212, 16609)

In [29]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score

# fit the training dataset on the classifier
import seaborn as sns
model = MLPClassifier()
model.fit(tf_v_matrix_train, y_train)
# predict the labels on validation dataset
y_pred_mlp = model.predict(tf_v_matrix_valid)

print(f"(percision score is :{precision_score(y_true = y_test, y_pred= y_pred_mlp, pos_label='positive', average='macro')}")
print(f"(accuracy score is :{accuracy_score(y_test, y_pred_mlp)}")






(percision score is :0.7035741098241097
(accuracy score is :0.7227722772277227


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [30]:
from sklearn.linear_model import LogisticRegression 
logreg = LogisticRegression(class_weight="balanced")
logreg.fit(tf_v_matrix_train, y_train) 
y_pred_class = logreg.predict(tf_v_matrix_valid)

print(f"(percision score is :{precision_score(y_true = y_test, y_pred= y_pred_class, pos_label='positive', average='macro')}")
print(f"(accuracy score is :{accuracy_score(y_test, y_pred_class)}")

(percision score is :0.7557514498080535
(accuracy score is :0.768976897689769


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [31]:
from sklearn import svm
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
model = svm.SVC(kernel='linear')
model.fit(tf_v_matrix_train, y_train)
# predict the labels on validation dataset
y_pred_svm = model.predict(tf_v_matrix_valid)
# Use accuracy_score function to get the accuracy
print(f"(percision score is :{precision_score(y_true = y_test, y_pred= y_pred_svm, pos_label='positive', average='macro')}")
print(f"(accuracy score is :{accuracy_score(y_test, y_pred_svm)}")




(percision score is :0.7421503593145384
(accuracy score is :0.7557755775577558


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [32]:
import seaborn as sns 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score


knn_model=KNeighborsClassifier(n_neighbors=1)

knn_model.fit(tf_v_matrix_train,y_train)

y_pred_knn=knn_model.predict(tf_v_matrix_valid)

print(f"(percision score is :{precision_score(y_true = y_test, y_pred= y_pred_knn, pos_label='positive', average='macro')}")
print(f"(accuracy score is :{accuracy_score(y_test, y_pred_knn)}")




(percision score is :0.6801609848484848
(accuracy score is :0.6963696369636964


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import  accuracy_score
classifierh = RandomForestClassifier(n_estimators=1000, random_state=0)
classifierh.fit(tf_v_matrix_train,y_train) 
y_pred_rfc = classifierh.predict(tf_v_matrix_valid)

print(f"(percision score is :{precision_score(y_true = y_test, y_pred= y_pred_rfc, pos_label='positive', average='macro')}")
print(f"(accuracy score is :{accuracy_score(y_test, y_pred_rfc)}")
# print(classification_report(y_test,y_pred_rfc))


(percision score is :0.6840131886087768
(accuracy score is :0.7029702970297029


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1370: UserWarning: Note that pos_label (set to 'positive') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
